In [1]:
!pip install transformers
!pip install numpy
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import pipeline
import textwrap
import numpy as np
import pandas as pd
from pprint import pprint


c:\Users\nihaltm\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
df = pd.read_csv("Team_15_Dataset_Stock_Market_Analysis.csv")

In [4]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Titles
0,2019-03-13,184.449997,184.899994,180.399994,181.699997,181.135452,9769821.0,NaN
1,2019-03-14,181.500000,182.449997,177.649994,179.550003,178.992142,15737080.0,Geneva 2019: Tata Motors To Reveal New Generat...
2,2019-03-15,180.199997,181.399994,177.600006,180.000000,179.440735,17490993.0,"Not just Tata Motors, a lot of other companies..."
3,2019-03-17,NaN,NaN,NaN,NaN,NaN,NaN,Geneva 2019: Tata Motors To Reveal New Generat...
4,2019-03-18,181.000000,184.699997,179.500000,182.250000,181.683746,14690339.0,NaN


In [5]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [16]:
example = wrap(df['Titles'][1])
print(example)

Geneva 2019: Tata Motors To Reveal New Generation Passenger Vehicles.
Tata Harrier receives over 10000 bookings in India since launch.  Not
just Tata Motors, a lot of other companies are excited by Jaguar Land
Rover numbers.


In [17]:
summarizer = pipeline('summarization')      # Uses distilbart-cnn-12-6 model by default (Abstractive summarization technique)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [23]:
prompt = summarizer(wrap(df['Titles'][1]))[0]['summary_text']
print(prompt)

Your max_length is set to 142, but your input_length is only 52. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


 Tata Harrier receives over 10000 bookings in India since launch . Jaguar Land Rover receives over 10,000 bookings since launch in India . Tata Motors to unveil new generation of vehicles in Geneva, Geneva, Germany, Switzerland next month . Tata also set to unveil a new vehicle in Geneva next week .


In [28]:
# INCOMPLETE

from transformers import pipeline
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

sentences = [title for title in df['Titles'].fillna('')]
# sentences = [prompt]
model_outputs = classifier(sentences)
print(model_outputs[0])

[{'label': 'neutral', 'score': 0.9549601674079895}, {'label': 'approval', 'score': 0.022920329123735428}, {'label': 'disapproval', 'score': 0.009044044651091099}, {'label': 'realization', 'score': 0.008665132336318493}, {'label': 'annoyance', 'score': 0.006453514564782381}, {'label': 'disappointment', 'score': 0.0055939024314284325}, {'label': 'admiration', 'score': 0.004978000186383724}, {'label': 'sadness', 'score': 0.0023011553566902876}, {'label': 'optimism', 'score': 0.002142630284652114}, {'label': 'confusion', 'score': 0.0020893877372145653}, {'label': 'excitement', 'score': 0.0019051289418712258}, {'label': 'joy', 'score': 0.0016943629598245025}, {'label': 'anger', 'score': 0.0015344731509685516}, {'label': 'disgust', 'score': 0.0014689210802316666}, {'label': 'gratitude', 'score': 0.0013705701567232609}, {'label': 'amusement', 'score': 0.001089825644157827}, {'label': 'fear', 'score': 0.0010508557315915823}, {'label': 'desire', 'score': 0.0010161612881347537}, {'label': 'curio